In [20]:
import os
import numpy as np
from keras.preprocessing import image
from keras.applications import densenet
import keras
import pickle
from scipy.spatial.distance import cosine
from PIL import Image


In [21]:
model = densenet.DenseNet201(include_top=False, weights='imagenet', input_shape=(256, 256, 3),pooling='avg',classes= 358)

In [22]:
def getencodings(path):    
    _img = image.load_img(path, target_size=(256, 256))
    _img = keras.utils.img_to_array(_img)
    _img = np.expand_dims(_img, axis=0)
    _enc = densenet.preprocess_input(_img)
    _enc = model.predict(_enc)
    return _enc

In [ ]:
if __name__ == '__main__':
    train_folder=r"C:\Users\Aarush Raj\OneDrive\Desktop\img2rec\DEEP-CHEF-PROJECT\downloaded_images\train"
    test_folder=r"C:\Users\Aarush Raj\OneDrive\Desktop\img2rec\DEEP-CHEF-PROJECT\downloaded_images\test"
    recipes_list=os.listdir(train_folder)
    recipes_list.sort(key=lambda item:int(item.split("_")[0]))
    train_names_list=os.listdir(train_folder)
    test_names_list=os.listdir(test_folder)
    encodings_list = []
    recipe_list=[]
    c = 0
    for i in range(len(recipes_list)):
        name=recipes_list[i]
        recipe_name=name.split("_")[1]
        recipe_index=name.split("_")[0]
        train_path=os.path.join(train_folder,name)
        train_names_list=os.listdir(train_path)
        for train_name in train_names_list:
            image_path=os.path.join(train_path,train_name)
            encoding = getencodings(image_path)
            encodings_list.append(encoding)
            recipe_list.append(name)
        test_path=os.path.join(test_folder,name)
        test_names_list=os.listdir(test_path)    
        for test_name in test_names_list:
            image_path=os.path.join(test_path,test_name)
            encoding = getencodings(image_path)
            encodings_list.append(encoding)
            recipe_list.append(name)
    print(len(recipe_list), len(encodings_list))

    with open(r'C:\Users\Aarush Raj\OneDrive\Desktop\img2rec\DEEP-CHEF-PROJECT\aarush\encodings.txt', 'wb') as file:
        pickle.dump(encodings_list, file)
    with open(r'C:\Users\Aarush Raj\OneDrive\Desktop\img2rec\DEEP-CHEF-PROJECT\aarush\enc_names.txt', 'wb') as file:
        pickle.dump(recipe_list, file)

just checking

In [ ]:
_img = image.load_img(r"C:\Users\Aarush Raj\OneDrive\Desktop\img2rec\DEEP-CHEF-PROJECT\downloaded_images\test\1_Naan\9_Naan.jpg", target_size=(256, 256))
_img = keras.utils.img_to_array(_img)
_img = np.expand_dims(_img, axis=0)
_enc = densenet.preprocess_input(_img)
_enc = model.predict(_enc)
_enc

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step


array([[9.7090713e-05, 7.5560465e-04, 1.5090167e-03, ..., 2.7459392e-01,
        1.9984721e-01, 1.2217317e-01]], dtype=float32)

In [ ]:
print(os.listdir(r"C:\Users\Aarush Raj\OneDrive\Desktop\img2rec\DEEP-CHEF-PROJECT\downloaded_images"))

['hello', 'test', 'train']


just checking

In [ ]:
img = image.load_img(r"C:\Users\Aarush Raj\OneDrive\Desktop\img2rec\DEEP-CHEF-PROJECT\downloaded_images\train\1_Naan\2_Naan.jpg", target_size=(256, 256))
img = keras.utils.img_to_array(img)
img = np.expand_dims(img, axis=0)
enc = densenet.preprocess_input(img)
enc = model.predict(enc)
print(enc)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 291ms/step
[[9.1914306e-05 5.8829499e-04 1.2086974e-03 ... 7.5686830e-01
  6.8234848e-03 7.4211019e-04]]


In [ ]:
img1 = image.load_img(r"C:\Users\Aarush Raj\OneDrive\Desktop\img2rec\DEEP-CHEF-PROJECT\downloaded_images\train\1_Naan\5_Naan.jpg", target_size=(256, 256))
img1 = keras.utils.img_to_array(img1)
img1 = np.expand_dims(img1, axis=0)
enc1 = densenet.preprocess_input(img1)
enc1 = model.predict(enc1)
print(enc1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 222ms/step
[[1.3076351e-04 3.2538065e-04 1.4036527e-03 ... 5.8270401e-01
  1.1677598e-02 9.1031656e-02]]


In [ ]:
similarity=cosine(_enc[0],enc[0])
s1=cosine(_enc[0],enc1[0])
s2=cosine(enc[0],enc1[0])
s3=cosine(enc1[0],enc[0])
print(similarity,s1,s2,s3)

0.38520339248235214 0.39427060395575675 0.38686255295183825 0.38686255295183825


In [ ]:
img7 = image.load_img(r"C:\Users\Aarush Raj\OneDrive\Desktop\img2rec\DEEP-CHEF-PROJECT\downloaded_images\train\1_Naan\7_Naan.jpg", target_size=(256, 256))
img7 = keras.utils.img_to_array(img7)
img7 = np.expand_dims(img7, axis=0)
enc7 = densenet.preprocess_input(img7)
enc7 = model.predict(enc7)
print(enc7)

img3 = image.load_img(r"C:\Users\Aarush Raj\OneDrive\Desktop\img2rec\DEEP-CHEF-PROJECT\downloaded_images\train\1_Naan\3_Naan.jpg", target_size=(256, 256))
img3 = keras.utils.img_to_array(img3)
img3 = np.expand_dims(img3, axis=0)
enc3 = densenet.preprocess_input(img3)
enc3 = model.predict(enc3)
print(enc3)

print(cosine(enc3[0],enc7[0]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step
[[6.5320186e-05 4.3773686e-04 1.4675669e-03 ... 1.0696392e+00
  2.8892964e-02 3.2143086e-02]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 288ms/step
[[1.1759455e-04 4.8694725e-04 1.9708984e-03 ... 1.1663135e+00
  9.2119738e-02 1.7724106e-02]]
0.16504474276782277


In [ ]:
print(type(_enc))
print(len(_enc[0]))
print(len(enc[0]))
print(_enc[0][156])

<class 'numpy.ndarray'>
1920
1920
1.6126285e-05


In [12]:
with open(r"C:\Users\Aarush Raj\OneDrive\Desktop\img2rec\DEEP-CHEF-PROJECT\aarush\enc_names.txt",encoding="utf-8",errors="ignore") as f:
    d=f.readline()
    print(d)

I      ](0_Chicken Tikka Masalahhhhhhhhh1_Naanhhhhhhhhh2_Chicken Curryhhhhhhhhh3_Aloo Gobihhhhhhhh4_Crock Pot Chicken Vindaloohhhhhhhhh5_Potato Curry Chickpeashhhhhhhhh6_Channa Masalahhhhhhhhh)7_Chicken Makhani (Indian Butter Chicken)8_Spicy Shrimp Coconut Milkh	h	h	h	h	h	h	h	h	9_Palak Paneerh



In [6]:
import os
lis=os.listdir(r"C:\Users\Aarush Raj\OneDrive\Desktop\img2rec\DEEP-CHEF-PROJECT\downloaded_images\train")
rec=[x.split("_")[0] for x in lis]
print(lis[0:5])
print(rec[0:5])


['0_Chicken Tikka Masala', '100_Indian Fry Bread', '101_Shrimp Vindaloo', '102_Masala chai', '103_Basmati Pulao nuts']
['0', '100', '101', '102', '103']


In [1]:
from encoder import getrecipes
l=getrecipes(r"C:\Users\Aarush Raj\OneDrive\Desktop\img2rec\DEEP-CHEF-PROJECT\downloaded_images\train\9_Palak Paneer\8_Palak Paneer.jpg")
print(l[0].strip("_")[1])

1/1 ━━━━━━━━━━━━━━━━━━━━ 14s 14s/step
_


In [4]:
print(l[0].split("_")[1])

Palak Paneer


In [7]:
from PIL import Image
name=l[0]
recipe_name=name.split("_")[1]
image_link="C:\\Users\\Aarush Raj\\OneDrive\\Desktop\\img2rec\\DEEP-CHEF-PROJECT\\downloaded_images\\train\\" + name + "\\1_" + recipe_name + ".jpg"
print(image_link)
image=Image.open(image_link)
image.show()

C:\Users\Aarush Raj\OneDrive\Desktop\img2rec\DEEP-CHEF-PROJECT\downloaded_images\train\9_Palak Paneer\1_Palak Paneer.jpg
